In [1]:
import numpy as np
import utils 

# Resampling 
### Resampling fatto a 192Khz in una nuova cartella mantenendo la struttura del dataset originale 
Vanno copiato i file di metadata nella nuova cartella?

In [3]:
import librosa
import soundfile as sf
import os

def resample_audio_files(audio_files, target_sr, output_dir='resampled'):
    # Crea la directory di output se non esiste
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file in audio_files:
        # Carica il file audio
        y, sr = librosa.load(file, sr=None, mono=True)

        # Fai il resampling del segnale audio alla nuova frequenza di campionamento
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

        # Ottieni il percorso relativo del file
        relative_path = os.path.relpath(file, 'Dataset')

        # Crea la directory di output corrispondente se non esiste
        output_subdir = os.path.join(output_dir, os.path.dirname(relative_path))
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        # Ottieni il nome del file senza l'estensione
        base_name = os.path.splitext(os.path.basename(file))[0]

        # Salva il segnale audio resampled nella directory di output come file WAV
        output_file = os.path.join(output_subdir, f'{base_name}_resampled.wav')
        sf.write(output_file, y_resampled, target_sr)

# Ottieni la lista dei file audio
audio_files_resampled = utils.get_audio_files('Dataset/df_paths.csv')

# Fai il resampling di tutti i file audio a 192kHz
resample_audio_files(audio_files_resampled, 192000)

In [4]:
df_paths, df_names = utils.create_dataframe_from_files("resampled")

In [7]:
#TODO: analisi per il resembling shannon Nyquist 
#TODO: da modificare --> non prendere i duplicati 
frequenciesTarget,frequenciesNonTarget = utils.get_frequencies('resampled/df_paths.csv')
    
min = np.min(frequenciesTarget)
max = np.max(frequenciesTarget)

print(min, max)

min = np.min(frequenciesNonTarget)
max = np.max(frequenciesNonTarget)

print(min, max)


192000 192000
192000 192000
